In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("train_PDjVQMB.csv")
df.head()

,ID,Age,Gender,Income,Balance,Vintage,Transaction_Status,Product_Holdings,Credit_Card,Credit_Category,Is_Churn
0,84e2fcc9,36,Female,5L - 10L,563266.44,4,0,1,0,Average,1
1,57fea15e,53,Female,Less than 5L,875572.11,2,1,1,1,Poor,0
2,8df34ef3,35,Female,More than 15L,701607.06,2,1,2,0,Poor,0
3,c5c0788b,43,Female,More than 15L,1393922.16,0,1,2,1,Poor,1
4,951d69c4,39,Female,More than 15L,893146.23,1,1,1,1,Good,1


In [3]:
df.drop('ID',axis=1,inplace=True)
df.head()

,Age,Gender,Income,Balance,Vintage,Transaction_Status,Product_Holdings,Credit_Card,Credit_Category,Is_Churn
0,36,Female,5L - 10L,563266.44,4,0,1,0,Average,1
1,53,Female,Less than 5L,875572.11,2,1,1,1,Poor,0
2,35,Female,More than 15L,701607.06,2,1,2,0,Poor,0
3,43,Female,More than 15L,1393922.16,0,1,2,1,Poor,1
4,39,Female,More than 15L,893146.23,1,1,1,1,Good,1


In [4]:
col = ["Gender","Credit_Category"]
dummy_df = pd.get_dummies(df[col],drop_first=True)

In [5]:
df[list(dummy_df.columns)] = dummy_df
df.drop(col,axis=1,inplace=True)
df.head()

,Age,Income,Balance,Vintage,Transaction_Status,Product_Holdings,Credit_Card,Is_Churn,Gender_Male,Credit_Category_Good,Credit_Category_Poor
0,36,5L - 10L,563266.44,4,0,1,0,1,0,0,0
1,53,Less than 5L,875572.11,2,1,1,1,0,0,0,1
2,35,More than 15L,701607.06,2,1,2,0,0,0,0,1
3,43,More than 15L,1393922.16,0,1,2,1,1,0,0,1
4,39,More than 15L,893146.23,1,1,1,1,1,0,1,0


In [6]:
df["Credit_Card"] = df["Credit_Card"].apply(lambda x: -1 if x==0 else 1)
df["Transaction_Status"] = df["Transaction_Status"].apply(lambda x: -1 if x==0 else 1)

In [220]:
def change_age(x):
    if(x in range(20,30)):
        return 1
    elif (x in range(30,40)):
        return 2
    elif (x in range(40,60)):
        return 3
    else:
        return 4
df["Age"] = df["Age"].apply(change_age)
df["Age"].value_counts()

3    3091
2    2555
1     679
4     325
Name: Age, dtype: int64

In [7]:
check_income= {"Less than 5L":1,
"5L - 10L":2,   
"10L - 15L":3,
"More than 15L":4
}
df["Income"] = df["Income"].apply(lambda x: check_income[x.strip()])
df["Income"].value_counts()

3    1885
2    1847
1    1573
4    1345
Name: Income, dtype: int64

In [8]:
holdings = {
    "1":1,
    "2":2,
    "3+":3
}
df["Product_Holdings"] = df["Product_Holdings"].apply(lambda x: holdings[x])
df["Product_Holdings"].dtypes

dtype('int64')

In [9]:
df["IncomeRatio"] = df["Balance"]/df["Income"]

In [10]:
y = df.pop("Is_Churn")
X = df

In [12]:
from imblearn.combine import SMOTETomek
from collections import Counter
counter = Counter(y)
print(f"Before Train: {counter}")
smtom = SMOTETomek(random_state=139)
X, y = smtom.fit_resample(X,y)
counter = Counter(y)
print(f"Before Train: {counter}")

Before Train: Counter({0: 3730, 1: 3730})
Before Train: Counter({0: 3576, 1: 3576})


In [24]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
X = scale.fit_transform(X)


In [25]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X,y,test_size=0.20,random_state=200)
print(f"train_set:{x_train.shape}{y_train.shape}")
print(f"Validation_set:{x_val.shape}{y_val.shape}")

train_set:(5721, 11)(5721,)
Validation_set:(1431, 11)(1431,)


In [26]:
# Support functions
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from scipy.stats import uniform

# Fit models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import xgboost as xgb
# Scoring functions
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [27]:
def best_model(model):
    print(model.best_score_)    
    print(model.best_params_)
    print(model.best_estimator_)
def get_auc_scores(y_actual, method,method2):
    auc_score = roc_auc_score(y_actual, method); 
    fpr_df, tpr_df, _ = roc_curve(y_actual, method2); 
    return (auc_score, fpr_df, tpr_df)

In [28]:
# Fit primal logistic regression
param_grid = {'C': [0.1,0.5,1,10,50,100], 'max_iter': [250], 'fit_intercept':[True],'intercept_scaling':[1],
              'penalty':['l2'], 'tol':[0.00001,0.0001,0.000001],'solver':["lbfgs","liblinear"]}
log_Grid = GridSearchCV(LogisticRegression(),param_grid, cv=10, refit=True, verbose=0)
log_Grid.fit(x_train,y_train)
best_model(log_Grid)

0.7089710028191704
{'C': 0.5, 'fit_intercept': True, 'intercept_scaling': 1, 'max_iter': 250, 'penalty': 'l2', 'solver': 'lbfgs', 'tol': 1e-05}
LogisticRegression(C=0.5, max_iter=250, tol=1e-05)


In [29]:
from sklearn.neighbors import KNeighborsClassifier
params = {
'n_neighbors':[5,6,7,8,9,10],
}
KNN_Grid = GridSearchCV(KNeighborsClassifier(),params, cv=10, refit=True, verbose=0)
KNN_Grid.fit(x_train,y_train)
best_model(KNN_Grid)

0.7484747800192826
{'n_neighbors': 10}
KNeighborsClassifier(n_neighbors=10)


In [30]:
# Fit random forest classifier
param_grid = {'max_depth': [3, 5, 6, 7, 8], 'max_features': [2,4,6,7,8,9],'n_estimators':[50,100],'min_samples_split': [3, 5, 6, 7]}
RanFor_grid = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, refit=True, verbose=0)
RanFor_grid.fit(x_train,y_train)
best_model(RanFor_grid)

0.7993378019360552
{'max_depth': 8, 'max_features': 2, 'min_samples_split': 7, 'n_estimators': 50}
RandomForestClassifier(max_depth=8, max_features=2, min_samples_split=7,
                       n_estimators=50)


In [31]:
print(classification_report(y_val, RanFor_grid.predict(x_val)))

              precision    recall  f1-score   support

           0       0.77      0.85      0.81       739
           1       0.82      0.73      0.78       692

    accuracy                           0.79      1431
   macro avg       0.80      0.79      0.79      1431
weighted avg       0.80      0.79      0.79      1431



In [242]:
print(classification_report(y_val, KNN_Grid.predict(x_val)))

              precision    recall  f1-score   support

           0       0.78      0.61      0.68       880
           1       0.70      0.84      0.76       954

    accuracy                           0.73      1834
   macro avg       0.74      0.73      0.72      1834
weighted avg       0.74      0.73      0.73      1834



In [243]:
print(classification_report(y_val, RanFor_grid.predict(x_val)))

              precision    recall  f1-score   support

           0       0.67      0.65      0.66       880
           1       0.69      0.71      0.70       954

    accuracy                           0.68      1834
   macro avg       0.68      0.68      0.68      1834
weighted avg       0.68      0.68      0.68      1834



In [31]:
def findGroupID(listPFR):
	sum = 0
	for i in range(0,listPFR_size):
		lst = []
		for j in range(i+1,listPFR_size):
			if listPFR[i]>listPFR[j]:
				lst.append("Greater")
			elif listPFR[i]<listPFR[j]:
				lst.append("Smaller")
			else:
				lst.append("Equal")
		if ("Greater" in lst) and ("Smaller" in lst):
			sum = sum+5
		elif ("Greater" in lst) and ("Smaller" not in lst):
			sum =sum+10
		else:
			sum = sum +15
	return sum

In [ ]:
arr = [4,2,5,7,1,3]

